<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E5%86%92%E9%9A%AA24_%E3%80%8A%E7%B4%85%E6%A8%93%E5%A4%A2%E3%80%8B%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入套件

In [2]:
import tensorflow as tf
import pickle

from tensorflow.keras.models import load_model, model_from_json

### 2. 讀入訓練好的 RNN 紅樓夢生成器

In [3]:
!wget --no-check-certificate \
    https://github.com/yenlung/Python-AI-Book/blob/main/dream_rnn.zip?raw=true \
    -O /content/dream_rnn.zip

--2022-08-21 01:49:58--  https://github.com/yenlung/Python-AI-Book/blob/main/dream_rnn.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/yenlung/Python-AI-Book/raw/main/dream_rnn.zip [following]
--2022-08-21 01:49:58--  https://github.com/yenlung/Python-AI-Book/raw/main/dream_rnn.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream_rnn.zip [following]
--2022-08-21 01:49:58--  https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream_rnn.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting resp

In [4]:
import os
import zipfile

local_zip = '/content/dream_rnn.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
f = open('architecture.json', 'r')
loaded_model = f.read()
f.close()

In [6]:
model = model_from_json(loaded_model)

In [7]:
model.load_weights("weights.h5")

In [8]:
f = open('tokenizer.pkl', 'rb')
tokenizer = pickle.load(f)
f.close()

#### 【已下載紅樓夢生成器請執行這段】

如果你已經在[我的 GitHub](https://github.com/yenlung/Deep-Learning-Basics) 中下載紅樓夢生成模型, 這包括:

1. `dream_rnn` 資料夾, 這是模型和訓練好的權重。
2. `dream_tokenizer2.pkl` 檔案, 這是模型使用的 tokenizer。

存到你的 Google Drive, 放 Colab 程式的地方 (預設是 `Colab Notebooks` 資料夾)。那執行下面這一段, 不用再讀入一次模型資料。

In [ ]:
#from google.colab import drive

#drive.mount('/content/drive')

In [ ]:
#%cd '/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
#model = load_model('dream_rnn')

In [ ]:
#f = open('dream_tokenizer2.pkl', 'rb')
#tokenizer = pickle.load(f)
#f.close()

### 3. 打造紅樓夢生成器

In [9]:
max_id = len(tokenizer.word_index)

In [10]:
max_id

4522

接下來是一段文字, 我們用事先訓練好的 tokenizer 換成一段數字, 最後用 one-hot encoding 回傳。

In [11]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences([texts]))-1
    return tf.one_hot(X, max_id)

這段程式主要依輸入的一段文字, 用我們的 model 去預測下一個字。注意像平常的分類問題, 這裡輸出是每人個字出現機率最高的。但都照這樣, 我們輸入同一段文字, 之後出現的文字永遠是一樣的! 常用的手法是去設定 `temperature`, `temperature` 接近 0, 大致上就取機率最高的字; `temperature` 越大就越隨機。太隨機就變成亂數取字! 一般 `temperature`設 1 左右效果最佳。

In [12]:
def next_char(texts, temperature=1):
    X_new = preprocess(texts)
    y_predict = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_predict) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

最後就一段文字進來，再產生 `n_chars` 這麼多個字。我們原本一段文字只能生一個字, 那就一次生一個字, 最後要生多少個字就生多少個字。

原本訓練我們一段是 100 個字去訓練的, 這裡超過 100 字時我們就取最後 100 個字丟入模型。

In [13]:
def complete_text(texts, n_chars=50, temperature=1):
    n_chars=int(n_chars)
    for _ in range(n_chars):
        texts = texts + next_char(texts[-100:], temperature)
    return texts

### 4. 測試紅樓夢生成器

In [14]:
complete_text("自孫悟空從石頭中蹦出來之後，", n_chars=300, temperature=0.2)

'自孫悟空從石頭中蹦出來之後，說道：「寶玉何嘗肯唸書，他便拿著那玉還抄去，便抄錄的《石頭記》給他看看。那知他的『假』字也不可犯前。那曹雪芹先生笑古史，那『香』字也不可犯。」那一僧說道：「果然是『寶玉』，那『假』字也不可沾染不是。」雨村聽了，益發驚異：「你們不知道士隱，何以此事？」士隱道：「你既此如此，恭恭敬敬作一個揖。我們老爺、太太都是那時候的，但是你們的心，如今得了清楚，才有這樣的好處，我是一定的。」寶釵道：「我也是這麼說，我是一句不知道的。」寶釵道：「我也不用說了，我是一句不知道的，我是一定的道理。」寶玉聽了，點頭嘆道：「我是個心酸起來，我是一句不知道的，也是不中用的，但只我這一句，你就不該慪人了。」說著，又把寶釵的話'

### 5. 用 `gradio` 做成一個網路 app

我們準備用 [`gradio`](https://gradio.app/) 套件, 神速做完一個 web app。最酷的是, 最後出現 `https://xxxx.gradio.app` 那個網址, 在你的 Colab 還在執行的時候, 任何人都可以用任何瀏覽器連進來使用!

In [15]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.1 MB 16.6 MB/s 
     |████████████████████████████████| 54 kB 4.4 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 57 kB 5.8 MB/s 
     |████████████████████████████████| 112 kB 62.6 MB/s 
     |████████████████████████████████| 212 kB 63.4 MB/s 
     |████████████████████████████████| 2.3 MB 62.0 MB/s 
     |████████████████████████████████| 275 kB 62.0 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 80 kB 12.7 MB/s 
     |████████████████████████████████| 68 kB 9.7 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 856 kB 56.

In [16]:
import gradio as gr

In [22]:
iface = gr.Interface(
    fn=complete_text,
    inputs=[
        "text",
        gr.Slider(minimum=50, maximum=200, value=50, step=1),
        gr.Slider(minimum=0.2, maximum=2, value=1, step=0.2)],
    outputs="text",
    title="紅樓夢生成器",
    description="起個頭, 幫你完成一段紅樓夢。可以改變 temperature, 越小生出的字越固定, 越大越隨機。")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://33758.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f56dfa59d10>,
 'http://127.0.0.1:7864/',
 'https://33758.gradio.app')